# Exercise: Exploratory Data Analysis with Pandas
## Instructions
1. Data Collection
    - Search and collect data on the following: (1993 - 2023)
        - Israel interest tares
        - US interest rates
        - USD to ILS exchange rates
    - For each dataset, create a pandas DataFrame with the following columns:
        - Year
        - Month
        - Current_Rate (Interest rate or exchange rate for that month)
        - Change_From_Last_Month (Difference in Current_Rate from the previous month)
        - Optional: You may include additional columns if you find them useful (e.g., Change_Percentage, Date).



In [1]:
# your code here
import numpy as np
import pandas as pd

In [41]:
israel_intrest_rate_dtypes = {
    'DATE': str,
    'IR3TIB01ILM156N': 'float64'
}
us_intrest_rate_dtypes = {
    'DATE': str,
    'IRLTLT01USM156N': 'float64'
}
usd_to_ils_exchange_rates_dtypes = {
    'DATE': str,
    'USD': 'float64'
}

In [42]:
israel_intrest_rate = pd.read_csv('./Israel interest rates.csv', dtype=israel_intrest_rate_dtypes, na_values=['nan', 'NaN', '', ' '], low_memory=False)
usd_to_ils_exchange_rates = pd.read_csv('./USD to ILS exchange rates.csv', dtype=usd_to_ils_exchange_rates_dtypes, na_values=['nan', 'NaN', '', ' '], low_memory=False)
us_intrest_rate = pd.read_csv('./US interest rates.csv', dtype=us_intrest_rate_dtypes, na_values=['nan', 'NaN', '', ' '], low_memory=False)

In [46]:
israel_intrest_rate['DATE'] = pd.to_datetime(israel_intrest_rate['DATE'])
us_intrest_rate['DATE'] = pd.to_datetime(us_intrest_rate['DATE'])
usd_to_ils_exchange_rates['DATE'] = pd.to_datetime(usd_to_ils_exchange_rates['DATE'], dayfirst=True)

In [65]:
# Israel intrest rate dataframe
israel_intrest_rate_df = pd.DataFrame({
    'Year': israel_intrest_rate['DATE'].dt.year,
    'Month': israel_intrest_rate['DATE'].dt.month,
    'Current_Rate': israel_intrest_rate['IR3TIB01ILM156N']
})

israel_intrest_rate_df['Change_From_Last_Month'] = israel_intrest_rate_df['Current_Rate'].diff()

israel_intrest_rate_df['Date'] = israel_intrest_rate['DATE']

israel_intrest_rate_df = israel_intrest_rate_df.dropna(subset=['Change_From_Last_Month'])

In [97]:
# US intrest rate dataframe
us_intrest_rate_df = pd.DataFrame({
    'Year': us_intrest_rate['DATE'].dt.year,
    'Month': us_intrest_rate['DATE'].dt.month,
    'Current_Rate': us_intrest_rate['IRLTLT01USM156N']
})

us_intrest_rate_df['Change_From_Last_Month'] = us_intrest_rate_df['Current_Rate'].diff()

us_intrest_rate_df['Date'] = us_intrest_rate['DATE']

us_intrest_rate_df['Change_Percentage'] = us_intrest_rate_df['Change_From_Last_Month'] / us_intrest_rate_df['Current_Rate'].shift(1) * 100

us_intrest_rate_df = us_intrest_rate_df.dropna(subset=['Change_From_Last_Month'])

In [126]:
# USD to ILS exchange rates dataframe
usd_to_ils_exchange_rates_df = pd.DataFrame({
    'Year': usd_to_ils_exchange_rates['DATE'].dt.year,
    'Month': usd_to_ils_exchange_rates['DATE'].dt.month,
    'Current_Rate': usd_to_ils_exchange_rates['USD']
})

usd_to_ils_exchange_rates_df['Change_From_Last_Month'] = usd_to_ils_exchange_rates_df['Current_Rate'].diff()

usd_to_ils_exchange_rates_df['Date'] = usd_to_ils_exchange_rates['DATE']

usd_to_ils_exchange_rates_df = usd_to_ils_exchange_rates_df.dropna(subset=['Change_From_Last_Month'])

## Part A: Single DataFrame Analysis 
1. Display the first 5 rows of the US interest rates DataFrame.

2. Calculate the average Israeli interest rate over the period covered by your data.

3. Find the month and year when the USD to ILS exchange rate had the highest increase from the previous month.

4. Determine the total number of months where the US interest rate decreased compared to the previous month.

5. Add a new column to the Israeli interest rates DataFrame called Change_Percentage, representing the percentage change from the previous month.



In [68]:
# A - 1
# your code here
us_intrest_rate_df.head(5)

,Year,Month,Current_Rate,Change_From_Last_Month,Date
1,1993,2,6.26,-0.34,1993-02-01
2,1993,3,5.98,-0.28,1993-03-01
3,1993,4,5.97,-0.01,1993-04-01
4,1993,5,6.04,0.07,1993-05-01
5,1993,6,5.96,-0.08,1993-06-01


In [69]:
# A - 2
# your code here
mean_current_rate = israel_intrest_rate_df['Current_Rate'].mean()
print(mean_current_rate)

5.105276612339456


In [70]:
# A - 3
# your code here
highest_change_from_mast_month = usd_to_ils_exchange_rates_df['Change_From_Last_Month'].max()
print(highest_change_from_mast_month)

0.14400000000000013


In [71]:
# A - 4
# your code here
count_of_decreasing_months = us_intrest_rate_df[us_intrest_rate_df['Change_From_Last_Month'] < 0].shape[0]
print(count_of_decreasing_months)

195


In [72]:
# A - 5
# your code here
israel_intrest_rate_df['Change_Percentage'] = israel_intrest_rate_df['Change_From_Last_Month'] / israel_intrest_rate_df['Current_Rate'].shift(1) * 100

## Part B: Merging DataFrames
1. Merge the US and Israeli interest rates DataFrames on Year and Month.

2. In the merged DataFrame, create a new column Rate_Difference that shows the difference between the Israeli and US interest rates for each month.

3. Identify all months when the Israeli interest rate was at least 1% higher than the US interest rate - IH (Israeli Higher).
Identify all months when the US interest rate was at least 1% higher than the Israeli interest rate - UH (US Higher).
Identify all months when the interest rates were within 1% of each other - N (Near)
make a new column called Rate_Comparison with the following values: IH, UH, or N.

4. Merge the exchange rate DataFrame with the merged interest rates DataFrame.

5. After merging all three DataFrames, find the correlation between the Rate_Difference and the Current_Rate of the exchange rate.
hint: find the connection between the difference in interest rates and the exchange rates, use any method you think is appropriate.
provide an explanation of the results (in Hebrew) - at least 2 rows of text + a proof of the connection(some output) you found.


In [119]:
# B - 1
# your code here
merged_us_and_israel_df = (pd.merge(israel_intrest_rate_df, us_intrest_rate_df, on='Date', how='left', suffixes=('_Israel', '_US'))
                           .drop(['Year_Israel', 'Month_Israel'], axis=1)
                           .rename(columns={"Year_US": "Year", "Month_US": "Month"}, errors="raise")
                           .fillna(0))
merged_us_and_israel_df

,Current_Rate_Israel,Change_From_Last_Month_Israel,Date,Change_Percentage_Israel,Year,Month,Current_Rate_US,Change_From_Last_Month_US,Change_Percentage_US
0,12.325500,0.778357,1993-02-01,0.000000,1993,2,6.26,-0.34,-5.151515
1,11.845000,-0.480500,1993-03-01,-3.898422,1993,3,5.98,-0.28,-4.472843
2,11.925333,0.080333,1993-04-01,0.678205,1993,4,5.97,-0.01,-0.167224
3,12.079524,0.154190,1993-05-01,1.292966,1993,5,6.04,0.07,1.172529
4,12.311364,0.231840,1993-06-01,1.919280,1993,6,5.96,-0.08,-1.324503
...,...,...,...,...,...,...,...,...,...
355,1.775618,0.381096,2022-09-01,27.328089,2022,9,3.52,0.62,21.379310
356,1.856479,0.080861,2022-10-01,4.553959,2022,10,3.98,0.46,13.068182
357,2.700107,0.843629,2022-11-01,45.442413,2022,11,3.89,-0.09,-2.261307
358,3.254688,0.554580,2022-12-01,20.539196,2022,12,3.62,-0.27,-6.940874


In [120]:
# B - 2
# your code here
merged_us_and_israel_df['Rate_Difference'] = (merged_us_and_israel_df['Current_Rate_US'] - merged_us_and_israel_df['Current_Rate_Israel']).abs()
merged_us_and_israel_df.head(10)

,Current_Rate_Israel,Change_From_Last_Month_Israel,Date,Change_Percentage_Israel,Year,Month,Current_Rate_US,Change_From_Last_Month_US,Change_Percentage_US,Rate_Difference
0,12.325500,0.778357,1993-02-01,0.000000,1993,2,6.26,-0.34,-5.151515,6.065500
1,11.845000,-0.480500,1993-03-01,-3.898422,1993,3,5.98,-0.28,-4.472843,5.865000
2,11.925333,0.080333,1993-04-01,0.678205,1993,4,5.97,-0.01,-0.167224,5.955333
3,12.079524,0.154190,1993-05-01,1.292966,1993,5,6.04,0.07,1.172529,6.039524
4,12.311364,0.231840,1993-06-01,1.919280,1993,6,5.96,-0.08,-1.324503,6.351364
5,11.117000,-1.194364,1993-07-01,-9.701311,1993,7,5.81,-0.15,-2.516779,5.307000
6,9.999565,-1.117435,1993-08-01,-10.051586,1993,8,5.68,-0.13,-2.237522,4.319565
7,10.228889,0.229324,1993-09-01,2.293336,1993,9,5.36,-0.32,-5.633803,4.868889
8,9.627895,-0.600994,1993-10-01,-5.875459,1993,10,5.33,-0.03,-0.559701,4.297895
9,10.003636,0.375742,1993-11-01,3.902635,1993,11,5.72,0.39,7.317073,4.283636


In [121]:
# B - 3
# your code here
conditions = [
   merged_us_and_israel_df['Change_Percentage_Israel'] > (merged_us_and_israel_df['Change_Percentage_US'] + 1),
   merged_us_and_israel_df['Change_Percentage_US'] > (merged_us_and_israel_df['Change_Percentage_Israel'] + 1),
   (merged_us_and_israel_df['Change_Percentage_Israel'] - merged_us_and_israel_df['Change_Percentage_US']).abs() < 1
]

choices = ['IH', 'UH', 'N']

merged_us_and_israel_df['Rate_Comparison'] = np.select(conditions, choices, default='Unknown')
merged_us_and_israel_df

,Current_Rate_Israel,Change_From_Last_Month_Israel,Date,Change_Percentage_Israel,Year,Month,Current_Rate_US,Change_From_Last_Month_US,Change_Percentage_US,Rate_Difference,Rate_Comparison
0,12.325500,0.778357,1993-02-01,0.000000,1993,2,6.26,-0.34,-5.151515,6.065500,IH
1,11.845000,-0.480500,1993-03-01,-3.898422,1993,3,5.98,-0.28,-4.472843,5.865000,N
2,11.925333,0.080333,1993-04-01,0.678205,1993,4,5.97,-0.01,-0.167224,5.955333,N
3,12.079524,0.154190,1993-05-01,1.292966,1993,5,6.04,0.07,1.172529,6.039524,N
4,12.311364,0.231840,1993-06-01,1.919280,1993,6,5.96,-0.08,-1.324503,6.351364,IH
...,...,...,...,...,...,...,...,...,...,...,...
355,1.775618,0.381096,2022-09-01,27.328089,2022,9,3.52,0.62,21.379310,1.744382,IH
356,1.856479,0.080861,2022-10-01,4.553959,2022,10,3.98,0.46,13.068182,2.123521,UH
357,2.700107,0.843629,2022-11-01,45.442413,2022,11,3.89,-0.09,-2.261307,1.189893,IH
358,3.254688,0.554580,2022-12-01,20.539196,2022,12,3.62,-0.27,-6.940874,0.365312,IH


In [185]:
# B - 4
# your code here
all_tables_merged = (pd.merge(merged_us_and_israel_df, usd_to_ils_exchange_rates_df, on='Date', suffixes=('', '_USD'))
                     .drop(['Year_USD', 'Month_USD'], axis=1)
                     .rename(columns={"Current_Rate": "Current_Rate_USD", "Change_From_Last_Month": "Change_From_Last_Month_USD"}, errors="raise")
                     .fillna(0))
all_tables_merged

,Current_Rate_Israel,Change_From_Last_Month_Israel,Date,Change_Percentage_Israel,Year,Month,Current_Rate_US,Change_From_Last_Month_US,Change_Percentage_US,Rate_Difference,Rate_Comparison,Current_Rate_USD,Change_From_Last_Month_USD
0,12.325500,0.778357,1993-02-01,0.000000,1993,2,6.26,-0.34,-5.151515,6.065500,IH,2.796,0.002
1,11.845000,-0.480500,1993-03-01,-3.898422,1993,3,5.98,-0.28,-4.472843,5.865000,N,2.796,0.004
2,11.925333,0.080333,1993-04-01,0.678205,1993,4,5.97,-0.01,-0.167224,5.955333,N,2.764,0.019
3,12.311364,0.231840,1993-06-01,1.919280,1993,6,5.96,-0.08,-1.324503,6.351364,IH,2.721,0.001
4,11.117000,-1.194364,1993-07-01,-9.701311,1993,7,5.81,-0.15,-2.516779,5.307000,UH,2.813,0.011
...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,0.364914,0.346624,2022-06-01,1895.092215,2022,6,3.14,0.24,8.275862,2.775086,IH,3.325,-0.015
223,0.850531,0.485617,2022-07-01,133.076940,2022,7,2.90,-0.24,-7.643312,2.049469,IH,3.526,0.030
224,1.394522,0.543990,2022-08-01,63.958909,2022,8,2.90,0.00,0.000000,1.505478,IH,3.382,0.015
225,1.775618,0.381096,2022-09-01,27.328089,2022,9,3.52,0.62,21.379310,1.744382,IH,3.364,-0.011


In [188]:
# B - 5
# your code here
correlation = all_tables_merged['Rate_Difference'].corr(all_tables_merged['Current_Rate_USD'])
print(correlation)

-0.46355266565168973


## Part C: Analyzing Relationships Between DataFrames
1. Analyze whether increases in US interest rates are associated with increases or decreases in the USD to ILS exchange rate in the same month.

2. Determine if changes in the Israeli interest rate have a stronger impact on the exchange rate than changes in the US interest rate.

3. Find any time lags in the effect of interest rate changes on the exchange rate. For example, does a change in US interest rates affect the exchange rate in the following month?
(כלומר- האם כשיש שינוי בריבית כלשהי - האם מייד יש שינוי בשער ההמרה? או שיש דיליי של חודש/חודשים עד שמרגישים את ההשפעה - אם בכלל?)

4. Calculate the rolling past 3-month average of the exchange rate for each data time, and add it as a new column.

5. Identify any periods where both US and Israeli interest rates were decreasing, and analyze how the exchange rate behaved during those periods.



In [193]:
# C - 1
correlation2 = all_tables_merged[['Change_From_Last_Month_US', 'Current_Rate_USD']].corr()
print(correlation2)

                           Change_From_Last_Month_US  Current_Rate_USD
Change_From_Last_Month_US                   1.000000          0.008882
Current_Rate_USD                            0.008882          1.000000


In [ ]:
# C - 2
correlation_israel = all_tables_merged[['Change_From_Last_Month_Israel', 'Current_Rate_USD']].corr().iloc[0, 1]

correlation_us = all_tables_merged[['Change_From_Last_Month_US', 'Current_Rate_USD']].corr().iloc[0, 1]

print(f"Correlation between Israeli interest rate change and USD to ILS exchange rate: {correlation_israel}")
print(f"Correlation between US interest rate change and USD to ILS exchange rate: {correlation_us}")

Correlation between Israeli interest rate change and USD to ILS exchange rate: -0.08740332925038896
Correlation between US interest rate change and USD to ILS exchange rate: 0.008882270530826514


In [ ]:
# C - 3
all_tables_merged['Change_From_Last_Month_US_Lag'] = all_tables_merged['Change_From_Last_Month_US'].shift(1)
all_tables_merged['Current_Rate_USD_Lag'] = all_tables_merged['Current_Rate_USD'].shift(1)

correlation_us_lag = all_tables_merged[['Change_From_Last_Month_US_Lag', 'Current_Rate_USD']].corr().iloc[0, 1]

correlation_israel = all_tables_merged[['Change_From_Last_Month_Israel', 'Current_Rate_USD']].corr().iloc[0, 1]

print(f"Correlation between US interest rate change (1-month lag) and USD to ILS exchange rate: {correlation_us_lag}")
print(f"Correlation between Israeli interest rate change and USD to ILS exchange rate: {correlation_israel}")

Correlation between US interest rate change (1-month lag) and USD to ILS exchange rate: -0.0016669953812479192
Correlation between Israeli interest rate change and USD to ILS exchange rate: -0.08740332925038896


In [ ]:
# Bonus 

# Bonus
choose 3 global / local events or periods that you think could have affected the exchange rate or interest rates
and analyze the effect of these events on the data you collected.
show proof of your analysis that supports your claim.

(כלומר- תבחנו אירועים שקרו בתקופה שאתם עובדים עליה ובדקו איך הם השפיעו על הנתונים שאתם עובדים איתם)
למשל: בחירות בארה״ב, בחירות בישראל, רוסיה אוקראינה, מלחמות בישראל וכו׳.

In [ ]:
# Your code here